<a href="https://colab.research.google.com/github/yichun-hub/FlickrCrawler-in-Tainan/blob/main/FlickrCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparation

#####Importing libraries and mounting Google Drive

In [ ]:
import json
import os, os.path
import csv
import requests
from datetime import timedelta, date, datetime
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#####Defining function

In [ ]:
#function to automatically generate the date range from the parameter
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
#function to help converting python dictionary to json file
def python_dict_to_json_file(file_path):
    try:
        # Get a file object with write permission.
        file_object = open(file_path, 'w')
        #dict_object = dict(name='Tom', age=25, score=100)
        # Save dict data into the JSON file.
        json.dump(raw_data, file_object)
        #print(file_path + ' created. ')
    except FileNotFoundError:
        print(file_path + ' not found. ')

#####Setting parameter for crawling

In [ ]:
root = '/content/drive/MyDrive/Colab Notebooks/Flickr'

#defining parameter for the API
api_key = 'c917012e5757b3103013e57f3cd78d54'
year = '2016'
minimum_taken_date = date(2016, 1, 1)
maximum_taken_date = date(2016, 12, 31)
lower_longitude = '120.033449'
lower_latitude = '22.887501'
upper_longitude = '120.656555'
upper_latitude = '23.413401'
bbox = lower_longitude+','+lower_latitude+','+upper_longitude+','+upper_latitude
# bbox_south_west_point = '115.05,8.866667'
# bbox_north_east_point = '115.3833,-8.433333'
accuracy = '12' #World level is 1, Country is ~3, Region is ~6, City is ~11, Street is ~16
has_geo = '1' #1 for geotagged photo, 0 for photo without geotag
step = 2 #range of days between each data crawling attempt

#defining date range for data
date_range = []
for single_date in daterange(minimum_taken_date, maximum_taken_date):
    single_date_string = single_date.strftime('%Y-%m-%d')
    date_range.append(single_date_string)
date_length = len(date_range)
print(date_length)
print('date range: ' + date_range[0] + ' - ' + date_range[-1])

365
date range: 2016-01-01 - 2016-12-30


#Data Crawling

#####Using 'photos.search' function from to collect data matching the defined parameter and exporting the result into JSON files

In [ ]:
url = 'https://api.flickr.com/services/rest/?method=flickr.photos.search'

i = 0
j = step - 1

while True:
    try:
        while j < date_length:
            params = dict(api_key=api_key,
                          format='json',
                          nojsoncallback='1',
                          min_taken_date=date_range[i],
                          max_taken_date=date_range[j],
                          bbox=bbox,
                          accuracy=accuracy,
                          has_geo=has_geo)
            resp = requests.get(url=url, params=params)
            data = json.loads(resp.text)

            file_object = open(root+'/JSON/photos.search/{0}/{1} - {2}.json' .format(year, date_range[i], date_range[j]), 'w')
            json.dump(data, file_object)

            print('{0} - {1}.json created.' .format(date_range[i], date_range[j]))
            # print(json.dumps(data, sort_keys=True, indent=4))
            i += step
            j += step
    except Exception as ex:
        print('{0} when trying to get data of {0} - {1}. Retrying...' .format(ex, date_range[i], date_range[j]))
        continue
    break

2018-01-01 - 2018-01-02.json created.
2018-01-03 - 2018-01-04.json created.
2018-01-05 - 2018-01-06.json created.
2018-01-07 - 2018-01-08.json created.
2018-01-09 - 2018-01-10.json created.
2018-01-11 - 2018-01-12.json created.
2018-01-13 - 2018-01-14.json created.
2018-01-15 - 2018-01-16.json created.
2018-01-17 - 2018-01-18.json created.
2018-01-19 - 2018-01-20.json created.
2018-01-21 - 2018-01-22.json created.
2018-01-23 - 2018-01-24.json created.
2018-01-25 - 2018-01-26.json created.
2018-01-27 - 2018-01-28.json created.
2018-01-29 - 2018-01-30.json created.
2018-01-31 - 2018-02-01.json created.
2018-02-02 - 2018-02-03.json created.
2018-02-04 - 2018-02-05.json created.
2018-02-06 - 2018-02-07.json created.
2018-02-08 - 2018-02-09.json created.
2018-02-10 - 2018-02-11.json created.
2018-02-12 - 2018-02-13.json created.
2018-02-14 - 2018-02-15.json created.
2018-02-16 - 2018-02-17.json created.
2018-02-18 - 2018-02-19.json created.
2018-02-20 - 2018-02-21.json created.
2018-02-22 -

#####Taking the 'photo_id', 'owner', and 'secret' from the previous JSON files and combining it into a python dictionaries list

In [ ]:
DIR = root+'/JSON/photos.search/{0}' .format(year)
photos_search_count = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
x = 0
i = 0
j = step - 1
dataset = []

while True:
    try:
        while x < photos_search_count:
            with open(root+'/JSON/photos.search/{0}/{1} - {2}.json' .format(year, date_range[i], date_range[j])) as data_file:
                n = 0
                jdata = json.load(data_file)
                jdata_len = len(jdata['photos']['photo'])
                while n < jdata_len:
                    new_corpus = {'id':jdata['photos']['photo'][n]['id'],
                                    'owner':jdata['photos']['photo'][n]['owner'],
                                    'secret':jdata['photos']['photo'][n]['secret']}
                    dataset.append(new_corpus)
                    n += 1
                #print('{0} corpus added from: {1} - {2}.json added.' .format(jdata_len, date_range[i], date_range[j]))
                x += 1
                i += step
                j += step
                n = 0
    except Exception as ex:
        print('{0} when trying to get data from to {1}. Retrying...' .format(ex, data_file))
    break

print('Total corpus added:', len(dataset))

Expecting value: line 1 column 1 (char 0) when trying to get data from to <_io.TextIOWrapper name='/content/drive/MyDrive/Colab Notebooks/Flickr/JSON/photos.search/2018/2018-12-29 - 2018-12-30.json' mode='r' encoding='UTF-8'>. Retrying...
Total corpus added: 6890


#####Getting detailed data of each photo using the 'getInfo' function and data from the previous dictioary and exporting it to JSON files

In [ ]:
url = 'https://api.flickr.com/services/rest/?method=flickr.photos.getInfo'
i = 0
dataset_length = len(dataset)
error_count = 0

while True:
    try:
        while i < dataset_length:
            params = dict(api_key=api_key,
                          format='json',
                          nojsoncallback='1',
                          photo_id=dataset[i]['id'],
                          secret=dataset[i]['secret'])
            resp = requests.get(url=url, params=params)
            data = json.loads(resp.text)
            # print(json.dumps(data, sort_keys=True, indent=4))

            file_object = open(root+'/JSON/getInfo/{0}/{1}.json' .format(year, dataset[i]['id']), 'w')
            json.dump(data, file_object)
            
            i += 1
            if i % 200 == 0:
                print('{0} / {1} JSON data created.' .format(i, dataset_length))
            error_count = 0
    except Exception as ex:
        error_count += 1
        print('{0} when trying to get data of {1}. Retrying...' .format(ex, dataset[i]['id']))
        if error_count > 5: pass
        else: continue
    break

print('Total JSON data created: {0}' .format(i))  

200 / 6890 JSON data created.
400 / 6890 JSON data created.
600 / 6890 JSON data created.
800 / 6890 JSON data created.
1000 / 6890 JSON data created.
1200 / 6890 JSON data created.
1400 / 6890 JSON data created.
1600 / 6890 JSON data created.
1800 / 6890 JSON data created.
2000 / 6890 JSON data created.
2200 / 6890 JSON data created.
2400 / 6890 JSON data created.
2600 / 6890 JSON data created.
2800 / 6890 JSON data created.
3000 / 6890 JSON data created.
3200 / 6890 JSON data created.
3400 / 6890 JSON data created.
3600 / 6890 JSON data created.
3800 / 6890 JSON data created.
4000 / 6890 JSON data created.
4200 / 6890 JSON data created.
4400 / 6890 JSON data created.
4600 / 6890 JSON data created.
4800 / 6890 JSON data created.
5000 / 6890 JSON data created.
5200 / 6890 JSON data created.
5400 / 6890 JSON data created.
5600 / 6890 JSON data created.
5800 / 6890 JSON data created.
6000 / 6890 JSON data created.
6200 / 6890 JSON data created.
6400 / 6890 JSON data created.
6600 / 6890 

#####Taking the 'photo_id', 'owner_NSID', 'owner_location', 'date_taken', and 'location' from the previous json files and combining it into a single JSON file

Note: Use [CodeBeautify](https://codebeautify.org/jsonviewer) to clean the JSON file and export it to csv format to make it easier to read and cleanse.

In [ ]:
i = 0
DIR = root+'/JSON/getInfo/{0}' .format(year)
file_list = [f for f in os.listdir(DIR) if '.json' in f]
num_files = len(file_list)
csv_columns = ['photo_id', 'owner_nsid', 'owner_location', 'dates_taken', 'location_latitude', 'location_longitude', 'description']
getinfo = []
print('Files found in the directory: ', num_files)

while True:
    try:
        # while i < num_files:
        for f in file_list:
            # with open(root+'/JSON/getInfo/{0}/{1}.json' .format(year, dataset[i]['id'])) as data_file:
            with open(root+'/JSON/getInfo/{0}/{1}' .format(year, f)) as data_file:
                jdata = json.load(data_file)
                new_corpus = {'photo_id':jdata['photo']['id'],
                              'owner_nsid':jdata['photo']['owner']['nsid'],
                              'owner_location':jdata['photo']['owner']['location'],
                              'dates_taken':jdata['photo']['dates']['taken'],
                              'location_latitude':jdata['photo']['location']['latitude'],
                              'location_longitude':jdata['photo']['location']['longitude'],
                              'description':jdata['photo']['description']['_content']}
                getinfo.append(new_corpus)
                #print('Corpus from: {0}.json is added' .format(dataset[i]['id']))
                i += 1
                if i % 500 == 0:
                    print('{0} / {1} files added to dictionary.' .format(i, num_files))
                    continue
    except Exception:
        print('{0} when trying to get data from to {1}' .format(Exception, f))
        # print('{0} when trying to get data from to {1}.json' .format(Exception, dataset[i]['id']))
        pass
    break

# print('{0} / {1} files added to dictionary.' .format(i, num_files))

try:
    with open(root+'/CSV/{0}_dirty.csv' .format(year), 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in getinfo:
            writer.writerow(data)
except IOError:
    print("I/O error")

print('A new CSV file containing {0} rows has been created.' .format(i))

Files found in the directory:  8605
500 / 8605 files added to dictionary.
1000 / 8605 files added to dictionary.
1500 / 8605 files added to dictionary.
2000 / 8605 files added to dictionary.
2500 / 8605 files added to dictionary.
3000 / 8605 files added to dictionary.
3500 / 8605 files added to dictionary.
4000 / 8605 files added to dictionary.
4500 / 8605 files added to dictionary.
5000 / 8605 files added to dictionary.
5500 / 8605 files added to dictionary.
6000 / 8605 files added to dictionary.
6500 / 8605 files added to dictionary.
7000 / 8605 files added to dictionary.
7500 / 8605 files added to dictionary.
8000 / 8605 files added to dictionary.
8500 / 8605 files added to dictionary.
A new CSV file containing 8605 rows has been created.


#Download image

In [ ]:
df = pd.read_csv(root+'/CSV/dirty/2019_dirty.csv')
df['description'] = ''

for i, row in df.iterrows():
    try:
        with open(root+'/JSON/getInfo/2019/{0}.json' .format(row['photo_id'])) as data_file: jdata = json.load(data_file)
    except FileNotFoundError:
        with open(root+'/JSON/getInfo/2020/{0}.json' .format(row['photo_id'])) as data_file: jdata = json.load(data_file)
    df.at[i, 'description'] = jdata['photo']['description']['_content']
    print(row['photo_id'], jdata['photo']['description']['_content'])

# df.to_csv(root+'/CSV/2019-2020_ready(1).csv')
# df.head()

photo_id = df['photo_id']

for idx in photo_id:
    with open(root+'/JSON/getInfo/{0}/{1}.json' .format(year, idx)) as data_file:    
        jdata = json.load(data_file)
    url = jdata['photo']['urls']['url'][0]['_content']
        
    html = urlopen(url)
    soup = BeautifulSoup(html)

    img = soup.find('img', {'class':'low-res-photo'})
    urllib.request.urlretrieve('https:'+img['src'], root+'/IMG/{0}/{1}_{2}.jpg' .format(year, jdata['photo']['dates']['taken'][:10], idx))

    if idx % 100 == 0: print('{0} / {1} images downloaded.' .format(i, len(photo_id)))

31610205537 
46632227401 ● Rider：<a href="https://pse.is/GKGDK" rel="noreferrer nofollow">傅校長</a>
● Photographer Team：<a href="https://pse.is/HJG7T" rel="noreferrer nofollow">Contact Dream 繫夢</a>
● Photographer：<a href="https://pse.is/E98HD" rel="noreferrer nofollow">Rui Huang 黃永睿</a>
● Camera：Canon EOS 6D + EF 24-70mm f/2.8L USM
46632216731 ● Rider：<a href="https://pse.is/GKGDK" rel="noreferrer nofollow">傅校長</a>
● Photographer Team：<a href="https://pse.is/HJG7T" rel="noreferrer nofollow">Contact Dream 繫夢</a>
● Photographer：<a href="https://pse.is/E98HD" rel="noreferrer nofollow">Rui Huang 黃永睿</a>
● Camera：Canon EOS 6D + EF 24-70mm f/2.8L USM
46632216001 ● Rider：<a href="https://pse.is/GKGDK" rel="noreferrer nofollow">傅校長</a>
● Photographer Team：<a href="https://pse.is/HJG7T" rel="noreferrer nofollow">Contact Dream 繫夢</a>
● Photographer：<a href="https://pse.is/E98HD" rel="noreferrer nofollow">Rui Huang 黃永睿</a>
● Camera：Canon EOS 6D + EF 24-70mm f/2.8L USM
46632215181 ● Rider：<a href="h